# Querying Satellite Data
This notebook uses content from the [Geospatial Carpentries](https://carpentries-incubator.github.io/geospatial-python/)

## ❓ Questions
-  Where can I find open-access satellite data?
-  How do I search for satellite imagery?
-  How do I fetch remote raster datasets using Python?


## ❗ Objectives
-  Search public STAC repositories of satellite imagery using Python.
-  Inspect a search result’s metadata.
-  Download (a subset of) the assets available for a satellite scene.
-  Open satellite imagery as raster data and save it to disk.

---

# Introduction
A number of satellites take snapshots of the Earth’s surface from space. The images recorded by these remote sensors represent a very precious data source for any activity that involves monitoring changes on Earth. 

Satellite imagery is typically provided in the form of geo-spatial raster data, with the measurements in each grid cell (“pixel”) being associated to accurate geographic coordinate information.

In this lesson we will explore how to access open satellite data using Python. In particular, we will consider the Sentinel-2 data collections hosted at SARA and AWS. This dataset consists of multi-band optical images acquired by the two satellites of the Sentinel-2 mission and it is continuously updated with new images.

## API's
An API is an Application Programming Interface.   

It is a way of having one application talk (interface) with another application in a pre-defined way. For what we're doing, these will be using web addresses and JSON data, and will be handled by our library, `pystac-client`.   

We will:
1. First initialise `pystac-client`.
2. Give it the information it needs.
3. Ask it to send send the request.
4. Investigate the results
5. Download some of the results

A useful resource will be the [pystac-client documentation](https://pystac-client.readthedocs.io/en/stable/quickstart.html).


# Initial setup
Some parameters we'll need throughout the lesson.  
> **GOOGLE COLAB**  
> If you're on google colab, the variable in this cell will already have been set in the top-cells, so just comment it out below with a `#` like so:
> ```py
> # storage_location = '../workshop_data'
> ```
> Alternatively delete the cell, or just dont run it!

If you're not on clab, run the cell by holding `Shift` then pressing `Enter`!

In [ ]:
# If you're on google colab, DONT RUN THIS CELL!
storage_location = '../workshop_data'

# Initialising PySTAC
The version of pystac below should print out to 0.8.2

In [ ]:
from pystac_client import Client
import pystac_client

print(pystac_client.__version__)

To use pystac_client, we need to inialise an API. We do so by defining a base url below.  



In [ ]:
api_url = "https://earth-search.aws.element84.com/v1"
client = Client.open(api_url)

In the following, we ask for scenes belonging to the sentinel-2-l2a collection. This dataset includes Sentinel-2 data products pre-processed at level 2A (bottom-of-atmosphere reflectance) and saved in Cloud Optimized GeoTIFF (COG) format:

In [ ]:
collection = "sentinel-2-l2a"  # Sentinel-2, Level 2A, Cloud Optimized GeoTiffs (COGs)

We also ask for scenes intersecting a geometry defined using the shapely library (in this case, a point):

In [ ]:
from shapely.geometry import Point
point = Point(116.06, -31.87)

Note: at this stage, we are only dealing with metadata, so no image is going to be downloaded yet. But even metadata can be quite bulky if a large number of scenes match our search! For this reason, we limit the search result to 10 items:

In [ ]:
search = client.search(
    collections=[collection],
    intersects=point,
    max_items=10,
    datetime=["2023-08-18", "2023-10-10"]
)

We submit the query and find out how many scenes match our search criteria (please note that this output can be different as more data is added to the catalog):

In [ ]:
print(search.matched())

Finally, we retrieve the metadata of the search results:

In [ ]:
items = search.item_collection()

The variable items is an ItemCollection object. We can check its size by:

In [ ]:
print(len(items))

which is consistent with the maximum number of items that we have set in the search criteria. We can iterate over the returned items and print these to show their IDs:

In [ ]:
for item in items:
    print(item)
    print(item.assets["thumbnail"].href)

Select a good image with little cloud cover by changing the `chosen_index` variable in the cell below
> **Refresher:** Negative index numbers look backwards, so `-1` is the last entry, and `-2` the second last, etc

In [ ]:
from pprint import pprint
chosen_index = -1
item = items[chosen_index]
print(item.datetime)
pprint(item.geometry)
pprint(item.properties)

# ✏ Exercise: Search satellite scenes using metadata filters


Search for all the available Sentinel-2 scenes in the sentinel-2-l2a collection that satisfy the following criteria: - intersect a provided bounding box (use ±0.01 deg in lat/lon from the previously defined point); - have been recorded between 20 March 2020 and 30 March 2020; - have a cloud coverage smaller than 10% (hint: use the query input argument of client.search).

How many scenes are available? Save the search results in GeoJSON format.

Hint: Buffer the previous point, and use the `bbox` search parameters


# Access the assets
So far we have only discussed metadata - but how can one get to the actual images of a satellite scene (the “assets” in the STAC nomenclature)? These can be reached via links that are made available through the item’s attribute assets.

In [ ]:
assets = items[chosen_index].assets  # first item's asset dictionary
print(assets.keys())

We can print a minimal description of the available assets:

In [ ]:
for key, asset in assets.items():
    print(f"{key}: {asset.title}")

In [ ]:
print(assets["thumbnail"].href)

Remote raster data can be directly opened via the rioxarray library. We will learn more about this library in the next episodes.

In [ ]:
import rioxarray
nir_href = assets["nir"].href
nir = rioxarray.open_rasterio(nir_href)
print(nir)

We can then save the data to disk:

In [ ]:
# save whole image to disk
import os 

nir_fpath = os.path.join(storage_location, "S2", "nir.tif")
os.makedirs(os.path.dirname(nir_fpath), exist_ok=True)

# nir.rio.to_raster(nir_fpath)

Since that might take a while, given there are over 10000 x 10000 = a hundred million pixels in the 10 meter NIR band, you can take a smaller subset before downloading it. Becuase the raster is a COG, we can download just what we need!

Here, we specify that we want to download the first (and only) band in the tif file, and a slice of the width and height dimensions.

In [ ]:
nir[0,2200:3200,1000:2200].rio.to_raster(nir_fpath)

The difference is 155 Megabytes for the large image vs about 4 Megabytes for the subset.

# Other bands
While we're at it, let's also download some other useful bands

In [ ]:
band_name = "red"
band_href = assets[band_name].href
band_xr = rioxarray.open_rasterio(band_href)

band_fpath = os.path.join(storage_location, "S2", f"{band_name}.tif")
band_xr[0, 2200:3200, 1000:2200].rio.to_raster(band_fpath)

In [ ]:
band_name = "visual"
band_href = assets[band_name].href
band_xr = rioxarray.open_rasterio(band_href)

band_fpath = os.path.join(storage_location, "S2", f"{band_name}.tif")
band_xr[:, 2200:3200, 1000:2200].rio.to_raster(band_fpath)

In [ ]:
band_xr[:, 2200:3200, 1000:2200].plot.imshow()

# Public catalogs, protected data

Publicly accessible catalogs and STAC endpoints do not necessarily imply publicly accessible data. Data providers, in fact, may limit data access to specific infrastructures and/or require authentication. For instance, the NASA CMR STAC endpoint considered in the last exercise offers publicly accessible metadata for the HLS collection, but most of the linked assets are available only for registered users (the thumbnail is publicly accessible).

The authentication procedure for datasets with restricted access might differ depending on the data provider. For the NASA CMR, follow these steps in order to access data using Python:

- Create a NASA Earthdata login account [here](https://urs.earthdata.nasa.gov/);
- Set up a netrc file with your credentials, e.g. by using this [script](https://git.earthdata.nasa.gov/projects/LPDUR/repos/daac_data_download_python/browse/EarthdataLoginSetup.py);
- Define the following environment variables:
```
import os
os.environ["GDAL_HTTP_COOKIEFILE"] = "./cookies.txt"
os.environ["GDAL_HTTP_COOKIEJAR"] = "./cookies.txt"
```



> **NOTE FOR ADVANCED USAGE:** 
> 
> While we use AWS earth-search in this lesson, there are many other cog-compliant URLS.    
> Quite a few are provided by Geoscience-Australia's DEA for their sandbox environment (but are accessible outside of it). These API urls can be found at:  
> https://explorer.sandbox.dea.ga.gov.au/products  
> Just note that not all parameters will always be shared amongst API's, and you may need to do filtering yourself
> I beleive currently they are set to owner-pays for data usage, and that they make no claims this will be permanent. Before relying on these API's for uptime-critical applications, please check any relevant information.    
> Status and uptime for these DEA URLS can be found [here](https://monitoring.dea.ga.gov.au/)

# Other options
Another valid choice for downloading satellite data using python is [eodag](https://eodag.readthedocs.io/en/stable/). 

PySTAC takes a bit more knowhow of API's to use, but gives you more freedom as well.  
PySTAC also supports COGs, which are currently only supported by the bleeding edge `eodag-cube` library rather than `eodag` itself.  
SARA and NCI (the locations we'd get the satellite data from for eodag) supply Sentinel-2 data as zip-files rather than COGs.